In [1]:
import os

import numpy as np
import cv2
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.losses import Loss
from scipy import ndimage
from sklearn.model_selection import train_test_split

physical_devices = tf.config.experimental.list_physical_devices('GPU')
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
config = tf.config.experimental.set_memory_growth(physical_devices[0], True)

# physical_devices = tf.config.experimental.list_physical_devices('GPU')
# assert len(physical_devices) > 0, "Not enough GPU hardware devices available"

# for i in range(len(physical_devices)):
#     config = tf.config.experimental.set_memory_growth(physical_devices[i], True)

In [2]:
data_size = 500
train_data = np.zeros((data_size,300,300,3), dtype=np.float32)

for i in range(0, data_size):
    img = keras.preprocessing.image.load_img('final_img/' + str(i) + '.png')
    img_arr = keras.preprocessing.image.img_to_array(img)
    
    train_data[i] = img_arr/255
    
data = pd.read_csv('reaug_data.csv')
pos = data[['ct_px','ct_py','st_px','st_py','ed_px','ed_py','pt_px','pt_py']][:data_size].to_numpy()

pos = pos.astype(np.float32)

In [3]:
#conv only
def gauge_recognition():
    inputs = keras.Input(shape=(300, 300, 3), batch_size=None)
    
    x = layers.Conv2D(filters=128, kernel_size=(3,3), padding='same')(inputs)
    x = layers.MaxPool2D()(x)
    x = layers.Activation('relu')(x)
    
    x = layers.Conv2D(filters=128, kernel_size=(3,3), padding='same')(x)
    x = layers.MaxPool2D()(x)
    x = layers.Activation('relu')(x)
    
    x = layers.Conv2D(filters=128, kernel_size=(3,3), padding='same')(x)
    x = layers.MaxPool2D()(x)
    x = layers.Activation('relu')(x)
    
    x = layers.Conv2D(filters=128, kernel_size=(3,3), padding='same')(x)
    x = layers.MaxPool2D()(x)
    x = layers.Activation('relu')(x)
    
    x = layers.Conv2D(filters=128, kernel_size=(3,3), padding='same')(x)
    x = layers.MaxPool2D()(x)
    x = layers.Activation('relu')(x)
    
    x = layers.Conv2D(filters=128, kernel_size=(3,3), padding='same')(x)
    x = layers.MaxPool2D()(x)
    x = layers.Activation('relu')(x)
    
    x = layers.Conv2D(filters=128, kernel_size=(3,3), padding='same')(x)
    x = layers.MaxPool2D()(x)
    x = layers.Activation('relu')(x)
    
    x = layers.Conv2D(filters=128, kernel_size=(2,2), padding='same')(x)
    x = layers.MaxPool2D()(x)
    x = layers.Activation('relu')(x)
    
    x = tf.keras.layers.Flatten()(x)
    
    x = layers.Dense(512)(x)
    x = layers.Dense(256)(x)
    x = layers.Dense(128)(x)
    x = layers.Dense(64)(x)
    x = layers.Dense(32)(x)
    x = layers.Dense(16)(x)
    outputs = layers.Dense(8)(x)
    
    
    model = keras.Model(inputs=inputs, outputs=outputs, name='gauge_recognition')
    
    return model

In [4]:
class CircluarMSE(Loss):
    def __init__(self):
        super().__init__()
        self.weight = 100
    
    def l2_distance(self, val1, val2):
        return tf.math.sqrt(tf.math.square(val1[:,0] - val2[:,0]) + tf.math.square(val1[:,1] - val2[:,1])) 
    
    def call(self, y_true, y_pred):
        #어차피 (None,8) 데이터 올것
        
        st_sim = tf.keras.losses.mean_squared_error(self.l2_distance(y_true[:,0:2],y_true[:,2:4]), 
                                                    self.l2_distance(y_pred[:,0:2],y_pred[:,2:4]))
        
        ed_sim = tf.keras.losses.mean_squared_error(self.l2_distance(y_true[:,0:2],y_true[:,4:6]), 
                                                    self.l2_distance(y_pred[:,0:2],y_pred[:,4:6]))
        
        pt_sim = tf.keras.losses.mean_squared_error(self.l2_distance(y_true[:,0:2],y_true[:,6:8]), 
                                                    self.l2_distance(y_pred[:,0:2],y_pred[:,6:8]))
        
        loss = st_sim + ed_sim + pt_sim + tf.keras.losses.mean_squared_error(y_true, y_pred)
        
        return loss

In [5]:
# strategy = tf.distribute.MultiWorkerMirroredStrategy()
# with strategy.scope():
model = gauge_recognition()

In [6]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0005),
              loss=CircluarMSE(), metrics=['mse', 'mae'])
model.fit(train_data[:data_size], pos[:data_size], epochs=50, batch_size=8)

Epoch 1/50
63/63 [==============================] - 12s 67ms/step - loss: 10309.7461 - mse: 7161.3955 - mae: 69.8761
Epoch 2/50
63/63 [==============================] - 4s 56ms/step - loss: 1911.4689 - mse: 1435.5260 - mae: 29.6060
Epoch 3/50
63/63 [==============================] - 4s 56ms/step - loss: 1323.4958 - mse: 903.1793 - mae: 22.3827
Epoch 4/50
63/63 [==============================] - 4s 56ms/step - loss: 1165.3818 - mse: 759.6223 - mae: 20.2789
Epoch 5/50
63/63 [==============================] - 4s 56ms/step - loss: 985.9345 - mse: 654.3615 - mae: 18.4654
Epoch 6/50
63/63 [==============================] - 4s 56ms/step - loss: 959.0331 - mse: 653.9253 - mae: 18.4973
Epoch 7/50
63/63 [==============================] - 3s 55ms/step - loss: 960.6144 - mse: 649.1066 - mae: 18.5004
Epoch 8/50
63/63 [==============================] - 4s 56ms/step - loss: 837.7167 - mse: 594.1594 - mae: 17.2273
Epoch 9/50
63/63 [==============================] - 4s 56ms/step - loss: 853.3743 - mse:

In [8]:
model.predict(train_data[0:1])

array([[127.02736, 118.36733,  90.0166 , 158.39038, 162.36392, 156.19872,
         75.94934, 108.34983]], dtype=float32)

In [15]:
pos[0]

array([125.0289 , 112.259  ,  89.33627, 154.02419, 159.57047, 148.97867,
        74.3182 , 105.73653], dtype=float32)

In [7]:
model.fit(train_data[:data_size], pos[:data_size], epochs=50, batch_size=8)

Epoch 1/50
63/63 [==============================] - 4s 56ms/step - loss: 85.0201 - mse: 52.7191 - mae: 5.5243
Epoch 2/50
63/63 [==============================] - 4s 56ms/step - loss: 77.9691 - mse: 50.6737 - mae: 5.3841
Epoch 3/50
63/63 [==============================] - 3s 55ms/step - loss: 63.9988 - mse: 41.6260 - mae: 4.8568
Epoch 4/50
63/63 [==============================] - 4s 56ms/step - loss: 55.8780 - mse: 35.9532 - mae: 4.5051
Epoch 5/50
63/63 [==============================] - 3s 55ms/step - loss: 75.4603 - mse: 42.9955 - mae: 4.9763
Epoch 6/50
63/63 [==============================] - 3s 55ms/step - loss: 55.0587 - mse: 33.3292 - mae: 4.4196
Epoch 7/50
63/63 [==============================] - 3s 55ms/step - loss: 42.7605 - mse: 26.8498 - mae: 3.9308
Epoch 8/50
63/63 [==============================] - 3s 55ms/step - loss: 46.5890 - mse: 26.7476 - mae: 3.9137
Epoch 9/50
63/63 [==============================] - 3s 55ms/step - loss: 47.0203 - mse: 28.6831 - mae: 4.1237
Epoch 10/5